In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### install & import

In [2]:
import numpy as np
import pandas as pd
import os

In [4]:
#os.chdir("/content/drive/MyDrive/소캡디/code")
os.chdir("/content/drive/MyDrive/4-1/소캡디/code")

### load data

##### evaluation_data & weight

In [5]:
eval_data = pd.read_pickle("./data_use/test_CA_1.pkl")

In [6]:
eval_data

,item_id,dept_id,cat_id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,2,0,1,0,0,1,4,...,2,1,2,0,0,1,0,1,3,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,0,2,0,1,0,1,0,...,1,0,0,1,0,0,2,1,1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,0,0,0,0,0,1,0,...,1,3,2,1,0,2,1,0,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,0,0,1,0,6,3,3,...,3,3,4,2,1,6,3,1,4,3
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,2,0,1,1,2,4,0,...,0,1,2,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823,FOODS_3,FOODS,0,5,1,0,0,4,0,...,2,3,5,1,2,1,5,1,2,2
3045,FOODS_3_824,FOODS_3,FOODS,1,1,0,1,3,1,0,...,0,3,0,3,1,0,6,0,0,1
3046,FOODS_3_825,FOODS_3,FOODS,1,0,2,0,1,1,2,...,3,1,1,1,2,2,0,0,0,0
3047,FOODS_3_826,FOODS_3,FOODS,5,0,3,0,5,0,0,...,1,2,1,3,6,3,2,1,4,3


In [7]:
weights = pd.read_csv("./data_init/weights_evaluation.csv")

In [8]:
weights

,Level_id,Agg_Level_1,Agg_Level_2,Dollar_Sales,weight
0,Level12,HOBBIES_1_001,CA_1,276.54,0.000071
1,Level12,HOBBIES_1_002,CA_1,27.79,0.000007
2,Level12,HOBBIES_1_003,CA_1,62.37,0.000016
3,Level12,HOBBIES_1_004,CA_1,227.36,0.000058
4,Level12,HOBBIES_1_005,CA_1,112.32,0.000029
...,...,...,...,...,...
42835,Level3,WI_3,X,323220.05,0.082856
42836,Level2,CA,X,1721795.38,0.441376
42837,Level2,TX,X,1076515.09,0.275960
42838,Level2,WI,X,1102665.55,0.282664


##### insample data

In [9]:
data = pd.read_pickle("./data_use/CA_1_final_grid_df.pkl")

In [10]:
data = data[['item_id','dept_id','cat_id','d','sales']]

In [11]:
data = data.dropna(subset=['sales']) #test data 삭제
data.reset_index(drop=True)

,item_id,dept_id,cat_id,d,sales
0,HOBBIES_1_008,HOBBIES_1,HOBBIES,1,12.0
1,HOBBIES_1_009,HOBBIES_1,HOBBIES,1,2.0
2,HOBBIES_1_010,HOBBIES_1,HOBBIES,1,0.0
3,HOBBIES_1_012,HOBBIES_1,HOBBIES,1,0.0
4,HOBBIES_1_015,HOBBIES_1,HOBBIES,1,4.0
...,...,...,...,...,...
4788262,FOODS_3_823,FOODS_3,FOODS,1941,2.0
4788263,FOODS_3_824,FOODS_3,FOODS,1941,0.0
4788264,FOODS_3_825,FOODS_3,FOODS,1941,1.0
4788265,FOODS_3_826,FOODS_3,FOODS,1941,1.0


##### prediction data

In [12]:
pred_category = pd.read_pickle("./data_use/pred_data_1203_cat.pkl")

In [13]:
pred_store = pd.read_pickle("./data_use/pred_data_1207_store.pkl")

In [14]:
pred_department = pd.read_pickle("./data_use/pred_data_1203_dept.pkl")

In [15]:
pred_data = eval_data.copy()

In [16]:
#pred_data.iloc[:,3:] = pred_department.iloc[:,3:]+pred_store.iloc[:,3:]+pred_category.iloc[:,3:]
pred_data.iloc[:,3:] = pred_store.iloc[:,3:]+pred_category.iloc[:,3:]
#pred_data.iloc[:,3:] = pred_category.iloc[:,3:]

In [17]:
pred_data.iloc[:,3:] = pred_data.iloc[:,3:]/2

In [18]:
pred_data.to_csv("./data_use/pred_data.csv", index = False)

# WRMSSE 

### RMSSE

In [19]:
#대회 코드에서 관측치의 수(n)의 기준을 처음 sales 값이 0이 아닌 d를 기준으로 했다.
def cal_start(data):
  tmp = data[data["sales"]>0]
  s = tmp.groupby(['item_id'],as_index=False).d.min()
  s.rename(columns = {'d':'start'},inplace=True)
  result = pd.merge(data, s, how='left', on="item_id")
  result=result[result["d"]>=result["start"]]
  return result

In [20]:
level = 12

In [22]:
if level == 12: #product-store
  insample12 = cal_start(data)

  ### item_id 단위로 diff 계산 ###
  insample12['dif'] = insample12.groupby(['item_id']).sales.transform('diff')
  insample12.dropna(subset=['dif'],inplace=True)
  insample12.reset_index(drop=True)
  insample12["dif"] = insample12["dif"]**2

  ### item_id별 diff 평균 ###
  tmp12 = insample12.groupby(['item_id'], as_index=False).dif.mean()

  ### item_id 단위로 예측 error값 계산 ###
  err12= eval_data.iloc[:,:3]
  err = (pred_data.iloc[: , 3:] - eval_data.iloc[: , 3:]) ** 2
  err12["error"]=err.mean(axis=1)
  tmp12= tmp12.merge(err12, on=['item_id'],how='left')

  ### RMSSE ###
  tmp12["RMSSE"] = np.sqrt(tmp12["error"]/tmp12["dif"])

elif level == 3: #store (CA_1 하나에 대해서만 계산)
  insample3 = data.groupby(['d'], as_index=False).sales.sum()

  ### store_id 단위로 diff 계산 
  insample3['dif'] = insample3.sales.diff()
  insample3.dropna(inplace=True)
  insample3['dif'] = insample3["dif"]**2
  mean_dif = insample3['dif'].mean()

  ### store 단위로 예측 error값 계산 ###
  CA_1_pred = pred_data.iloc[: , 3:].sum().to_frame()
  CA_1_test = eval_data.iloc[:,3:].sum().to_frame()
  err3 = CA_1_test
  err3= (CA_1_pred-CA_1_test)**2
  error = err3.mean()

  ### RMSSE ###
  RMSSE = np.sqrt(error/mean_dif)

elif level == 8: #store-category
  insample8 = data.groupby(['d','cat_id'], as_index=False).sales.sum()
  insample8['dif'] = insample8.groupby(['cat_id']).sales.transform('diff')
  insample8.dropna(inplace=True)
  insample8['dif'] = insample8["dif"]**2
  tmp8 = insample8.groupby(['cat_id'], as_index=False).dif.mean()

  cat_pred = pred_data.groupby(['cat_id'],as_index=False).sum()
  cat_eval = eval_data.groupby(['cat_id'],as_index=False).sum()

  err8 = (cat_pred.iloc[: , 3:] - cat_eval.iloc[: , 3:]) ** 2
  tmp8["error"]= err8.mean(axis=1)
  tmp8["RMSSE"] = np.sqrt(tmp8["error"]/tmp8["dif"])


elif level == 9: #store-department
  insample9 = data.groupby(['d','dept_id'], as_index=False).sales.sum()
  insample9['dif'] = insample9.groupby(['dept_id']).sales.transform('diff')
  insample9.dropna(inplace=True)
  insample9['dif'] = insample9["dif"]**2
  tmp9 = insample9.groupby(['dept_id'], as_index=False).dif.mean()

  dept_pred = pred_data.groupby(['dept_id'],as_index=False).sum()
  dept_eval = eval_data.groupby(['dept_id'],as_index=False).sum()

  err9 = (dept_pred.iloc[: , 3:] - dept_eval.iloc[: , 3:]) ** 2
  tmp9["error"]= err9.mean(axis=1)
  tmp9["RMSSE"] = np.sqrt(tmp9["error"]/tmp9["dif"])

In [ ]:
RMSSE

0    0.33813
dtype: float64

In [23]:
tmp12["RMSSE"].mean()

0.6244554368931742

In [ ]:
tmp8["RMSSE"].mean()

0.43120271434954094

In [ ]:
tmp8

,cat_id,dif,error,RMSSE
0,FOODS,382110.863918,51821.032959,0.368263
1,HOBBIES,15790.147423,7874.423446,0.706181
2,HOUSEHOLD,36716.412371,1763.586252,0.219164


In [ ]:
tmp9

,dept_id,dif,error,RMSSE
0,FOODS_1,5817.199485,3319.696595,0.755427
1,FOODS_2,11709.557216,5634.790129,0.693695
2,FOODS_3,232240.274227,30014.899761,0.359501
3,HOBBIES_1,14991.024742,6739.284819,0.670488
4,HOBBIES_2,217.773196,281.647609,1.137237
5,HOUSEHOLD_1,23530.952062,1165.831984,0.222586
6,HOUSEHOLD_2,2182.861340,108.199366,0.222638


In [ ]:
tmp9["RMSSE"].mean()

0.5802244874359973

------------------------------------

##### test

In [ ]:
insample_8 = data.groupby(['d','cat_id'], as_index=False).sales.sum()
insample_8['dif'] = insample_8.groupby(['cat_id']).sales.transform('diff')
insample_8.dropna(inplace=True)
insample_8['dif'] = insample_8["dif"]**2

In [ ]:
tmp_8 = insample_8.groupby(['cat_id'], as_index=False).dif.mean()

In [ ]:
tmp_8

,cat_id,dif
0,FOODS,382110.863918
1,HOBBIES,15790.147423
2,HOUSEHOLD,36716.412371


In [ ]:
cat_pred = pred_data.groupby(['cat_id'],as_index=False).sum()

In [ ]:
cat_pred

,cat_id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,FOODS,2939.798578,2666.911320,2592.155098,2659.213628,3031.324345,3686.023588,3838.088878,3014.704777,2680.888415,...,3251.175258,4074.493789,4267.256025,3143.201432,2746.922313,2616.916464,2722.844323,3090.821198,3736.665295,3969.364583
1,HOBBIES,510.354193,483.289255,482.743590,493.052647,552.079245,648.614415,662.493525,513.514114,482.394988,...,572.715766,665.748080,682.543410,525.597654,492.870141,486.778215,491.234621,549.003655,645.998324,663.007079
2,HOUSEHOLD,919.197168,863.665302,849.673130,870.648384,973.790294,1178.510861,1254.059847,1108.537598,913.284690,...,1043.503841,1264.193876,1354.055792,1000.957669,873.663830,865.975657,844.544155,993.234443,1197.523382,1136.331077


In [ ]:
cat_eval = eval_data.groupby(['cat_id'],as_index=False).sum()

In [ ]:
cat_eval

,cat_id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,FOODS,2805,2592,2572,3129,3119,3442,3995,3316,2974,...,3189,3848,4164,3038,2760,2800,2702,3282,3866,3347
1,HOBBIES,416,494,566,480,638,527,513,514,550,...,642,637,664,570,428,560,556,469,657,509
2,HOUSEHOLD,870,852,833,872,941,1119,1259,1181,960,...,1015,1254,1436,1019,882,874,818,980,1157,1021


In [ ]:
err_8 = (cat_pred.iloc[: , 3:] - cat_eval.iloc[: , 3:]) ** 2

In [ ]:
err_8

,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,406.227981,220699.235655,7687.020497,59547.511546,24621.100161,90778.811438,85914.401281,55020.903969,38997.370272,4291.743183,...,3865.762727,51299.436252,10661.806689,11067.341398,171.025905,33519.581041,434.485802,36549.334472,16727.466002,387337.674533
1,6931.629760,170.371587,7382.376163,14790.065942,22348.313880,0.236085,4570.437705,29922.142644,20983.111420,2484.845789,...,4800.305116,826.452104,343.858063,1971.568370,4208.135162,5361.429806,4194.554321,6400.584740,121.036881,23718.180453
2,277.993256,1.826867,1075.203394,3541.542614,24.405115,5250.799684,2182.320220,6.579853,21.163118,283.613957,...,812.468925,103.915108,6714.853219,325.525703,69.491737,64.390077,704.592189,175.150474,1642.144495,13301.257237


In [ ]:
tmp_8["error"]= err_8.mean(axis=1)

In [ ]:
tmp_8

,cat_id,dif,error
0,FOODS,382110.863918,51821.032959
1,HOBBIES,15790.147423,7874.423446
2,HOUSEHOLD,36716.412371,1763.586252


In [ ]:
tmp_8["RMSSE"] = np.sqrt(tmp_8["error"]/tmp_8["dif"])


In [ ]:
tmp_8

,cat_id,dif,error,RMSSE
0,FOODS,382110.863918,51821.032959,0.368263
1,HOBBIES,15790.147423,7874.423446,0.706181
2,HOUSEHOLD,36716.412371,1763.586252,0.219164


In [ ]:
weights_3 = weights[weights["Level_id"]=="Level3"]

In [ ]:
weights_3

,Level_id,Agg_Level_1,Agg_Level_2,Dollar_Sales,weight
42826,Level3,CA_1,X,434031.39,0.111262
42827,Level3,CA_2,X,438764.72,0.112476
42828,Level3,CA_3,X,590417.97,0.151351
42829,Level3,CA_4,X,258581.30,0.066286
42830,Level3,TX_1,X,308179.08,0.079001
42831,Level3,TX_2,X,374765.07,0.096070
42832,Level3,TX_3,X,393570.94,0.100890
42833,Level3,WI_1,X,338198.78,0.086696
42834,Level3,WI_2,X,441246.72,0.113112
42835,Level3,WI_3,X,323220.05,0.082856


In [ ]:
RMSSE * 0.111262

0    0.161313
dtype: float64

##### WRMSSE

In [24]:
weights_12 = weights.loc[weights["Level_id"]=="Level12"]
weights_12 = weights_12.loc[weights["Agg_Level_2"]=="CA_1"]

In [25]:
tmp12["WRMSSE"] = 0
tmp12["WRMSSE"] = tmp12["RMSSE"]* weights_12["weight"]
tmp12["WRMSSE"].sum()

0.06393911843123029

In [ ]:
weights_8 = weights.loc[weights["Level_id"]=="Level8"]
weights_8 = weights_8.loc[weights["Agg_Level_1"]=="CA_1"]
weights_8 = weights_8.reset_index(drop=True)

In [ ]:
tmp8["WRMSSE"] = 0
tmp8["WRMSSE"] = tmp8["RMSSE"]* weights_8["weight"]
tmp8["WRMSSE"].sum()

0.04222525777701823

In [ ]:
weights_9 = weights.loc[weights["Level_id"]=="Level9"]
weights_9 = weights_9.loc[weights["Agg_Level_1"]=="CA_1"]
weights_9 = weights_9.reset_index(drop=True)

In [ ]:
tmp9["WRMSSE"] = 0
tmp9["WRMSSE"] = tmp9["RMSSE"]* weights_9["weight"]
tmp9["WRMSSE"].sum()

0.04872177095987536